In [1]:
import pyspark
from pyspark.sql import SparkSession
import os

# set jvm options
# os.environ["SPARK_SUBMIT_OPTS"] = "-Xmx4g -XX:+UnlockDiagnosticVMOptions -XX:+LogVMOutput -XX:LogFile=/tmp/jvm.log"

In [2]:
!pyspark --version
# spark.stop()

25/01/26 16:40:52 WARN Utils: Your hostname, mackutis.local resolves to a loopback address: 127.0.0.1; using 172.20.10.12 instead (on interface en0)
25/01/26 16:40:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.4
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 17.0.14
Branch HEAD
Compiled by user yangjie01 on 2024-12-17T04:51:46Z
Revision a6f220d951742f4074b37772485ee0ec7a774e7d
Url https://github.com/apache/spark
Type --help for more information.


In [4]:
!echo $PYSPARK_SUBMIT_ARGS

"--name" "PySparkShell" "pyspark-shell"


In [5]:
# Add the package dependency to run kafka
spark_version = "3.5.1"
scala_version = "2.12"
# kafka_version = 

# packages = [
#     f"com.datastax.spark:spark-cassandra-connector_{scala_version}:{spark_version}",
#     f"org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}"
#     f"org.apache.spark:spark-avro_{scala_version}:{spark_version}",
# ]
# packages_str = ",".join(packages)

# https://stackoverflow.com/questions/58723314/pyspark-failed-to-find-data-source-kafka/58723724#58723724
# https://stackoverflow.com/questions/31841509/pyspark-exception-java-gateway-process-exited-before-sending-the-driver-its-p
# os.environ["PYSPARK_SUBMIT_ARGS"] = f"--conf spark.default.parallelism=2 --packages {packages_str} pyspark-shell"
# print(os.environ["PYSPARK_SUBMIT_ARGS"])
spark = (
    SparkSession.builder
    # .master("http://172.19.0.3:7078")
    # .master("spark://localhost:7077")
    .appName("test_kafka_read_stream")
    .config("spark.cassandra.connection.host", "localhost:9042")
    .getOrCreate()
)

# spark.stop()

25/01/26 16:41:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
kafka = (
    spark
    .readStream
    .format("kafka")
    # .option("kafka.bootstrap.servers", "localhost:9092")
    .option("kafka.bootstrap.servers", "http://broker:29092")
    .option("subscribe", "page_view_events")
    .option("startingOffsets", "earliest")
    .load()
)

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.

In [ ]:
# from pyspark.sql.avro.functions import from_avro
from pyspark.sql.functions import (
    window, col, date_format, unix_timestamp, from_json, approx_count_distinct
)
# from pyspark.sql import SchemaConverters
from pyspark.sql.types import (
    StructType, StructField, IntegerType, StringType, LongType, DoubleType
)

schema = StructType([
    StructField('ts', LongType(), True),
    StructField('sessionId', IntegerType(), True),
    StructField('page', StringType(), True),
    StructField('auth', StringType(), True),
    StructField('method', StringType(), True),
    StructField('status', IntegerType(), True),
    StructField('level', StringType(), True),
    StructField('itemInSession', IntegerType(), True),
    StructField('city', StringType(), True),
    StructField('zip', StringType(), True),
    StructField('state', StringType(), True),
    StructField('userAgent', StringType(), True),
    StructField('lon', DoubleType(), True),
    StructField('lat', DoubleType(), True),
    StructField('userId', IntegerType(), True),
    StructField('lastName', StringType(), True),
    StructField('firstName', StringType(), True),
    StructField('gender', StringType(), True),
    StructField('registration', LongType(), True),
    StructField('artist', StringType(), True),
    StructField('song', StringType(), True),
    StructField('duration', DoubleType(), True),
])

# jsonFormatSchema = open("../eventsim/schemas/page_view_events.avsc", "r").read()
# jsonFormatSchema

In [ ]:
batch_counter = 0
max_batches = 5

def process_batch(df, epoch_id):
    global batch_counter
    batch_counter += 1
    print(f"Processing batch {batch_counter}")

    (
        df
        .selectExpr("CAST(value as STRING)")
        .select(from_json("value", schema=schema).alias('value'))
        .select("value.*")
    )
    # # Collect and print the original DataFrame
    # original_data = df.collect()
    # print("Original DataFrame:")
    # for row in original_data:
    #     print(row)

    # # Collect and print the DataFrame after casting value to STRING
    # df_str = df.selectExpr("CAST(value as STRING)")
    # string_data = df_str.collect()
    # print("DataFrame after casting value to STRING:")
    # string_data = string_data.show(truncate=False)
    # # for row in string_data:
    # #     print(row)

    # # Collect and print the DataFrame after parsing JSON
    # df_json = df_str.select(from_json(col("value"), schema).alias('value'))
    # json_data = df_json.collect()
    # print("DataFrame after parsing JSON:")
    # for row in json_data:
    #     print(row)

    if batch_counter >= max_batches:
        query.stop()

df_pageviews = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "page_view_events") \
    .load()

df_pageviews_str = df_pageviews.selectExpr("CAST(value as STRING) as value_string", "offset")

output = (
    df_pageviews_str
    # df_pageviews
    # .select("value")
    .select(from_json("value_string", schema=schema).alias('value_json'), "offset")
    .select("value_json.*", "offset")
    .selectExpr("timestamp_millis(ts) as ts_timestamp", "*")
    .selectExpr(
        "to_date(ts_timestamp) as date", 
        "month(ts_timestamp) as month",
        "year(ts_timestamp) as year",
        "hour(ts_timestamp) as hour",
        "minute(ts_timestamp) as minute",
        "second(ts_timestamp) as second", 
        "*")
    # .select("ts_timestamp", "offset")
)

output = (
    output
    .withWatermark("ts_timestamp", "60 seconds")
    .groupBy(
        window(output.ts_timestamp, "60 seconds", "10 seconds"),
        # "minute"
    )
    .count()
    # .agg(
    #     approx_count_distinct("sessionId").alias("sessions"),
    # )
    .withColumn("window_start", date_format(col("window.start"), "yyyy-MM-dd HH:mm:ss"))
    .withColumn("unix_timestamp", unix_timestamp(col("window_start")))
    .selectExpr("window_start as timestamp_minute", "unix_timestamp", "count")
    # .selectExpr("window", "window_start as timestamp_minute", "unix_timestamp", "count")
    # .sort(col("window"), ascending=True)
    # .count()
    # .selectExpr("COUNT(distinct sessionId) as session_count", "minute")
    # .select(count_distinct("sessionId").alias("session_count"), "minute")
)

query = (
    output
    # df_pageviews
    .writeStream
    # .foreachBatch(process_batch)
    .outputMode("update")
    .format("console")
    # .format("org.apache.spark.sql.cassandra")
    # .options(table="metrics", keyspace="spark_streaming")
    # .option("checkPointLocation", "/tmp/checkpoint/")
    # .options(table="page_views_test", keyspace="spark_streaming")
    # .option("truncate", "false")
    # .option("numRows", "2")
    # .trigger(processingTime="1 minute")
    .trigger(processingTime="60 seconds")
    .start()
)

In [12]:
query.stop()

# spark.stop()